# End-to-End Demonstration: Schoology

*Goal*: run the Schoology Extractor and then upload the results into an `LMS` database.

Execution time is highly dependent on the number of students and the number of sections in your installation of Schoology.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.

In [1]:
# Load some utilities
from IPython.display import display, Markdown
import ipywidgets as widgets

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed. Below that, there will be two runtime text input boxes to provide a Schoology OAuth key and secret. You can find your key and secret at [https://app.schoology.com/api](https://app.schoology.com/api).

In [2]:
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "s-data"
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433

In [3]:
def get_key_secret(key, secret):
    return key, secret

key_widget = widgets.Text(
    description="Key"
)
secret_widget = widgets.Text(
    description="Secret"
)
result = widgets.interactive(get_key_secret, key=key_widget, secret=secret_widget)
display(result)

interactive(children=(Text(value='', description='Key'), Text(value='', description='Secret'), Output()), _dom…

In [6]:
KEY = result.children[0].value
SECRET = result.children[1].value

## Run the Schoology Extractor

In [9]:
from edfi_schoology_extractor.helpers.arg_parser import MainArguments as s_args
from edfi_schoology_extractor import schoology_extract_facade

arguments = s_args(
    client_key=KEY,
    client_secret=SECRET,
    output_directory=OUTPUT_DIRECTORY,
    log_level=LOG_LEVEL,
    page_size=200,
    input_directory=None
)

schoology_extract_facade.run(arguments)

INFO:edfi_schoology_extractor.schoology_extract_facade:Starting Ed-Fi LMS Schoology Extractor
INFO:edfi_schoology_extractor.schoology_extract_facade:Exporting s-data\users\2021-04-21-09-02-21.csv
INFO:edfi_schoology_extractor.helpers.csv_writer:The file has been generated => s-data\users\2021-04-21-09-02-21.csv
INFO:edfi_schoology_extractor.schoology_extract_facade:Exporting s-data\sections\2021-04-21-09-02-58.csv
INFO:edfi_schoology_extractor.helpers.csv_writer:The file has been generated => s-data\sections\2021-04-21-09-02-58.csv
INFO:edfi_schoology_extractor.schoology_extract_facade:Exporting s-data\section=3267169944\assignments\2021-04-21-09-02-58.csv
... remaining messages have been cleared to keep this notebook from being too large


## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [10]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)

INFO:edfi_lms_ds_loader.loader_facade:Begin loading files into the LMS Data Store (DS)...
INFO:edfi_lms_ds_loader.migrator:Begin database auto-migration...
INFO:edfi_lms_ds_loader.migrator:Done with database auto-migration.
INFO:edfi_lms_ds_loader.df_to_db:Uploading LMSUser file ...
INFO:edfi_lms_ds_loader.df_to_db:Done with LMSUser file.
INFO:edfi_lms_ds_loader.df_to_db:Uploading LMSSection file ...
INFO:edfi_lms_ds_loader.df_to_db:Done with LMSSection file.
INFO:edfi_lms_ds_loader.df_to_db:Uploading Assignment file ...
INFO:edfi_lms_ds_loader.df_to_db:Done with Assignment file.
... remaining messages have been cleared to keep this notebook from being too large
